In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [5]:
with open("../tmp/pacbio4G_cc.txt.k16.min60.c0.005") as f:
    lines =f.readlines()
len(lines)

12666

In [6]:
lines=[u.strip().split(",") for u in lines]

In [8]:
counts=[len(u) for u in lines]
np.argmax(counts)

1378

In [16]:
maxcc=set([int(u) for u in lines[1378]])
len(maxcc)

556072

In [17]:
edges=pd.read_csv('../tmp/pacbio4G_edges.txt',header=None)
edges.columns=['src','dest','cnt']

In [18]:
edges.shape

(86967293, 3)

In [19]:
edges['src_in_maxcc']= edges['src'].map(lambda u: u in maxcc)

In [20]:
edges['dest_in_maxcc']= edges['dest'].map(lambda u: u in maxcc)

In [24]:
edges['maxcc'] =( edges['src_in_maxcc'] & edges['dest_in_maxcc'])

In [34]:
edges.head()

,src,dest,cnt,src_in_maxcc,dest_in_maxcc,maxcc
0,308510,1067999,21,False,True,False
1,854894,889578,43,True,True,True
2,990902,1080973,111,True,True,True
3,860230,860730,30,True,True,True
4,388542,573483,212,True,True,True


In [26]:
maxccedges=edges[edges['maxcc']]

In [30]:
maxccedges=maxccedges[['src','dest','cnt']]

In [31]:
maxccedges.shape

(73246201, 3)

In [32]:
73246201/556072.

131.7207142240573

In [35]:
maxccedges.head()

,src,dest,cnt
1,854894,889578,43
2,990902,1080973,111
3,860230,860730,30
4,388542,573483,212
5,616093,652710,48


In [44]:
reads=pd.read_csv("../tmp/pacbio4G.seq",header=None, sep='\t', usecols=[0,1])
reads.columns=['id','desc']
reads.shape

(1107889, 2)

In [46]:
mapped_edges=pd.merge(maxccedges,reads, how='left', right_on='id',left_on='src')

In [53]:
%time mapped_edges=pd.merge(mapped_edges,reads, how='left', right_on='id',left_on='dest')

CPU times: user 1min 29s, sys: 9.51 s, total: 1min 39s
Wall time: 1min 39s


In [55]:
mapped_edges=mapped_edges[mapped_edges.cnt>=60]

In [56]:
mapped_edges.shape

(27859358, 7)

In [59]:
results=mapped_edges[['desc_x','desc_y','cnt']]

In [60]:
results.columns=['src','dest','count']

In [61]:
results.head()

,src,dest,count
1,m141130_053540_42161_c100698142550000001823143...,m150129_081052_sherri_c10073081003000000182315...,111
3,m150116_105100_42142_c100769831910000001823165...,m150202_065211_42141_c100692750070000001823137...,212
8,m150118_063823_42161_c100769502230000001823165...,m150118_130430_42161_c100769502230000001823165...,107
11,m150115_233910_42142_c100769831910000001823165...,m150121_201238_42142_c100753522550000001823145...,1233
12,m150124_130209_42161_c100730732540000001823150...,m150124_172122_42161_c100730732540000001823150...,127


In [67]:
%time results.to_csv('../tmp/pacbio4G_edges4biggestCC.txt.k16.min60.c0.005.gz',index=None,sep='\t',compression='gzip')

CPU times: user 18min 2s, sys: 3.03 s, total: 18min 5s
Wall time: 18min 51s


In [70]:
maxccedges[maxccedges['cnt']>=60].to_csv('../tmp/pacbio4G_edges4biggestCC_onlynumid.txt.k16.min60.c0.005.gz',index=None,sep='\t',compression='gzip')